# Introduction to Automation with LangChain, Generative AI, and Python
**2.9: Persisting Langchain Memory**
* Instructor: [Jeff Heaton](https://youtube.com/@HeatonResearch), WUSTL Center for Analytics and Business Insight (CABI), [Washington University in St. Louis](https://olin.wustl.edu/faculty-and-research/research-centers/center-for-analytics-and-business-insight/index.php)
* For more information visit the [class website](https://github.com/jeffheaton/cabi_genai_automation).

In this section, we delve into the creation of an innovative utility class designed for managing chat interactions—aptly named ChatBot. This class stands as a cornerstone for applications that require robust handling of chat history, including features to load and save these interactions, as well as capabilities to undo or regenerate outputs. The architecture of ChatBot integrates advanced language models with structured templates to facilitate interactive conversations, ensuring a seamless and dynamic user experience.

The core functionality of ChatBot revolves around its sophisticated integration of components from the langchain ecosystem. It utilizes a large language model for generating chat responses and another for summarizing conversations, complemented by a template that structures each interaction. Key features include maintaining an editable conversation history, implementing undo and regenerate operations for greater control over chat flows, and mechanisms to persist conversation states, enhancing the utility and flexibility of the chat interface. As we explore ChatBot, we will uncover how each component contributes to its overall capability to manage complex conversational contexts effectively.

The code for this class follows.

In [1]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory
from langchain_aws import ChatBedrock
from langchain_core.prompts.chat import PromptTemplate
from IPython.display import display_markdown
import pickle

DEFAULT_TEMPLATE = """You are a helpful assistant. Format answers with markdown.

Current conversation:
{history}
Human: {input}
AI:"""

MODEL = 'anthropic.claude-3-sonnet-20240229-v1:0'

class ChatBot:
    def __init__(self, llm_chat, llm_summary, template):
        """
        Initializes the ChatBot with language models and a template for conversation.

        :param llm_chat: A large language model for handling chat responses.
        :param llm_summary: A large language model for summarizing conversations.
        :param template: A string template defining the conversation structure.
        """
        self.llm_chat = llm_chat
        self.llm_summary = llm_summary
        self.template = template
        self.prompt_template = PromptTemplate(input_variables=["history", "input"], template=self.template)

        # Initialize memory and conversation chain
        self.memory = ConversationSummaryMemory(llm=self.llm_summary)
        self.conversation = ConversationChain(
            prompt=self.prompt_template,
            llm=self.llm_chat,
            memory=self.memory,
            verbose=False
        )

        self.history = []

    def converse(self, prompt):
        """
        Processes a conversation prompt and updates the internal history and memory.

        :param prompt: The input prompt from the user.
        :return: The generated response from the language model.
        """
        self.history.append([self.memory.buffer, prompt])
        output = self.conversation.invoke(prompt)
        return output['response']

    def chat(self, prompt):
        """
        Handles the full cycle of receiving a prompt, processing it, and displaying the result.

        :param prompt: The input prompt from the user.
        """
        print(f"Human: {prompt}")
        output = self.converse(prompt)
        display_markdown(output, raw=True)

    def print_memory(self):
        """
        Displays the current state of the conversation memory.
        """
        print("**Memory:")
        print(self.memory.buffer)

    def clear_memory(self):
        """
        Clears the conversation memory.
        """
        self.memory.clear()

    def undo(self):
        """
        Reverts the conversation memory to the state before the last interaction.
        """
        if len(self.history) > 0:
            self.memory.buffer = self.history.pop()[0]
        else:
            print("Nothing to undo.")

    def regenerate(self):
        """
        Re-executes the last undone interaction, effectively redoing an undo operation.
        """
        if len(self.history) > 0:
            self.memory.buffer, prompt = self.history.pop()
            self.chat(prompt)
        else:
            print("Nothing to regenerate.")

    def save_history(self, file_path):
        """
        Saves the conversation history to a file using pickle.

        :param file_path: The file path where the history should be saved.
        """
        with open(file_path, 'wb') as f:
            pickle.dump(self.history, f)

    def load_history(self, file_path):
        """
        Loads the conversation history from a file using pickle.

        :param file_path: The file path from which to load the history.
        """
        with open(file_path, 'rb') as f:
            self.history = pickle.load(f)
            # Optionally reset the memory based on the last saved state
            if self.history:
                self.memory.buffer = self.history[-1][0]



## Testing the Chat Bot
Before we see how this class was constructed, lets see how you can use it. The following shows a simple chat interaction.

In [2]:
MODEL = 'anthropic.claude-3-sonnet-20240229-v1:0'

# Initialize bedrock, use built in role
llm = ChatBedrock(
        model_id=MODEL,
        model_kwargs={"temperature": 0.1},
)

c = ChatBot(llm, llm, DEFAULT_TEMPLATE)

c.chat("Hello, my name is Jeff.")
c.chat("I like coffee.")
c.chat("What is my name.")
c.chat("Do I like coffee?")




Human: Hello, my name is Jeff.


```
Nice to meet you, Jeff! I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you may have. Please let me know if there's anything I can assist with.
```

Human: I like coffee.


Great, I'm glad you enjoy coffee! Here are a few thoughts on coffee in markdown format:

### Coffee Facts

- **Origin**: Coffee is believed to have originated in the forests of Ethiopia, where coffee trees grew wild.

- **Popular Varieties**: Some of the most popular coffee varieties include Arabica, Robusta, Liberica, and Excelsa.

- **Caffeine Content**: A typical 8 oz cup of brewed coffee contains around 95mg of caffeine.

- **Preparation Methods**: Common ways to brew coffee include drip brewing, French press, pour over, espresso, and cold brew.

### Coffee Trivia

- The word "coffee" comes from the Arabic term "qahwah" which referred to a type of wine.

- Finland has the highest per capita coffee consumption in the world.

- Coffee is one of the most traded commodities in the world after oil.

Let me know if you have any other coffee-related questions! I'm always happy to discuss this beloved beverage.

Human: What is my name.


Based on the conversation, your name is Jeff.

```
Jeff: *introduces themselves as Jeff*
AI: *greets Jeff*
```

Human: Do I like coffee?


```
Based on our conversation, you did mention that you like coffee. Specifically, you said "I like coffee" after introducing yourself as Jeff.
```

## Load and Save Chat Memory

The chat bot allows the complete chat history and memory to be saved and loaded from a file.

In [3]:
# Initialize bedrock, use built in role
llm = ChatBedrock(
        model_id=MODEL,
        model_kwargs={"temperature": 0.1},
)

c = ChatBot(llm, llm, DEFAULT_TEMPLATE)
c.chat("Hello, my name is Jeff.")
c.chat("I like coffee.")
c.chat("What is my name.")
c.chat("Do I like coffee?")

Human: Hello, my name is Jeff.


```
Nice to meet you, Jeff! I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you may have. Please let me know if there's anything I can assist with.
```

Human: I like coffee.


Great, I'm glad you enjoy coffee! Here are a few thoughts on coffee in markdown format:

### Coffee Facts

- **Origin**: Coffee is believed to have originated in the forests of Ethiopia, where coffee trees grew wild.

- **Popular Varieties**: Some of the most popular coffee varieties include Arabica, Robusta, Liberica, and Excelsa.

- **Caffeine Content**: A typical 8 oz cup of brewed coffee contains around 95mg of caffeine.

- **Preparation Methods**: Common ways to brew coffee include drip brewing, French press, pour over, espresso, and cold brew.

### Coffee Trivia

- The word "coffee" comes from the Arabic term "qahwah" which referred to a type of wine.

- Finland has the highest per capita coffee consumption in the world.

- Coffee is one of the most traded commodities in the world after oil.

Let me know if you have any other coffee-related questions! I'd be happy to discuss further.

Human: What is my name.


Based on the conversation, your name is Jeff.

Human: Do I like coffee?


```
Based on the conversation, you stated that you like coffee:

We now save the chat memory.

In [4]:
c.save_history('history.pkl')

You can now reload the chat memory; the bot should remember the discussion.

In [5]:
c = ChatBot(llm, llm, DEFAULT_TEMPLATE)
c.load_history('history.pkl')
c.chat("What is my name?")

Human: What is my name?


Based on the conversation, your name is **Jeff**.

## Undoing and Regenerating the Chat

The chatbot allows us to undo parts of the conversation, and it forgets. We can also regenerate outputs to get a better response.

In [6]:
# Initialize bedrock, use built in role
llm = ChatBedrock(
        model_id=MODEL,
        model_kwargs={"temperature": 0.1},
)

c = ChatBot(llm, llm, DEFAULT_TEMPLATE)

c.chat("Hello, my name is Jeff.")
c.chat("I like coffee.")
c.undo()
c.chat("What is my name.")
c.chat("Do I like coffee?")
c.regenerate()



Human: Hello, my name is Jeff.


```
Nice to meet you, Jeff! I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you may have. Please let me know if there's anything I can assist with.
```

Human: I like coffee.


Great, I'm glad you like coffee! Here are a few thoughts on coffee in markdown format:

### Coffee Facts

- **Origin**: Coffee is believed to have originated in the forests of Ethiopia, where coffee trees grew wild.

- **Popular Varieties**: Some of the most popular coffee varieties include Arabica, Robusta, Liberica, and Excelsa.

- **Caffeine Content**: A typical 8 oz cup of brewed coffee contains around 95mg of caffeine.

- **Preparation Methods**: Common ways to brew coffee include drip brewing, French press, pour over, espresso, and cold brew.

### Coffee Trivia

- The word "coffee" comes from the Arabic term "qahwah" which referred to a type of wine.

- Finland has the highest per capita coffee consumption in the world.

- Coffee is one of the most traded commodities in the world after oil.

Let me know if you have any other coffee-related questions! I'm always happy to discuss this beloved beverage.

Human: What is my name.


```
Your name is Jeff, based on your introduction at the start of our conversation.
```

Human: Do I like coffee?


```
Unfortunately, I don't have enough information to know whether you like coffee or not. As an AI assistant, I don't have personal knowledge about your preferences unless you provide that information to me.
```

Human: Do I like coffee?


```
Unfortunately, I don't have enough information to know whether you like coffee or not. As an AI assistant without personal knowledge about you, I can only respond based on what you directly tell me.
```

## Implementing the Chatbot

In this section, we delve into the architecture of a sophisticated chatbot built using the LangChain and Bedrock libraries, encapsulated within the ChatBot class. The ChatBot class leverages two language models and a structured conversation template to engage users in a dynamic conversational flow.

The class initiates by defining its basic components: llm_chat and llm_summary are large language models used for chat responses and conversation summarization, respectively. The template parameter defines the conversation's structure, guiding how interactions are formatted and presented. This template incorporates markdown formatting and placeholders for the conversation history and the latest user input, facilitating a clear and structured dialogue presentation.

Upon instantiation, the ChatBot initializes various components. It constructs a PromptTemplate, which prepares the structure for input variables and the conversation template. The ConversationSummaryMemory, initialized with the llm_summary, acts as a dynamic storage for conversation excerpts, aiding in recalling context or summarizing past dialogues. This memory component is crucial for maintaining conversation continuity, especially in sessions that might span multiple interactions.

The ConversationChain is another pivotal element, combining the prompt template with the llm_chat model to process and respond to user inputs. The chain's verbose mode is set to False to streamline output without extraneous system messages.

Interaction with the ChatBot is primarily through the chat method, where users provide a prompt, and the bot displays the response after processing. Internally, the converse method updates the conversation history and uses the conversation chain to generate a response. This method's utility lies in its ability to append each interaction to the history log, ensuring all discourse components are retained for future reference or undo actions.

Memory management functions like print_memory, clear_memory, and undo provide users and developers with tools to view, reset, or revert the conversation state, thus offering flexibility in managing the dialogue flow and content. Additional functionalities to save and load conversation history are provided through save_history and load_history, utilizing Python’s pickle module for object serialization.

This class structure not only facilitates robust conversation handling and memory management but also showcases the integration of advanced language models in practical applications, demonstrating the potential of AI in enhancing interactive software solutions. Through such a detailed setup, ChatBot serves as a comprehensive framework for building interactive, memory-aware chatbot systems.

## Interactive Chat Session

The following code implements an interactive chat session with memory and rollback/regenerate.

In [28]:
from IPython.display import display, Markdown, HTML

c = ChatBot(llm, llm, DEFAULT_TEMPLATE)

while True:
    prompt = input("Human>>> ").strip()
    if prompt[0] == "/":
        cmd = prompt[1:].strip().lower()
        if cmd == 'bye':
            break
        elif cmd == 'undo':
            c.undo()
        elif cmd == 'redo':
            c.regenerate()
        elif cmd == 'dump':
            c.print_memory()
        else:
            display(HTML(f"<span style='color: red;'>Unknown command: {cmd}</span>"))
            
    else:
        response = c.converse(prompt)
        markdown_text = f"<span style='color: blue;'>AI>>> {response}</span>"
        display(Markdown(markdown_text))

display(HTML("<span style='color: red;'>Chat done.</span>"))

Human>>>  /redo


Nothing to regenerate.


Human>>>  /undo


Nothing to undo.


Human>>>  I like coffee


...

Human>>>  /undo
Human>>>  /undo


Nothing to undo.


Human>>>  /redo


Nothing to regenerate.


KeyboardInterrupt: Interrupted by user